# MLG HW3

In [11]:
import pandas as pd
import numpy as np
import os, glob, json
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import ndcg_score


import torch
from deepctr_torch.inputs import SparseFeat, get_feature_names
import torchfm



config = {'test_size' : 0.2, 
          'n_epoch' : 20}
rmse = {}
ndcg_10 = {}
recall_10 = {}


with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
# with open("./metrics/recall.txt", "r") as fp:
#     recall_10 = json.load(fp)
print(rmse)
print(ndcg_10)

temp = pd.read_csv('results.csv', header = [0, 1], index_col = [0])

{'bookIPNN': 0.7630446736162781, 'bookOPNN': 0.7596840523834419, 'bookPNN': 0.7518341524053948, 'bookCCPM': 0.7424491460611815, 'bookWDL': 0.7093089364684898, 'bookDCN': 0.7302810220618285, 'bookNFM': 0.7568314381827344, 'bookDeepFM': 0.7386060319249964, 'bookAFM': 0.7353113174115252, 'bookxDeepFM': 0.7524432051769194, 'movieIPNN': 1.1257715623148956, 'movieOPNN': 1.0723128454116133, 'moviePNN': 1.1457441626815474, 'movieCCPM': 1.053634406962656, 'movieWDL': 1.0492728323684728, 'movieDCN': 1.0530302980259754, 'movieNFM': 1.0942265873231904, 'movieDeepFM': 1.0515179004479969, 'movieAFM': 1.0710224032118236, 'moviexDeepFM': 1.0744275480292602, 'businessIPNN': 1.2039563606524462, 'businessOPNN': 1.1574377340755466, 'businessPNN': 1.1446729801067166, 'businessCCPM': 1.0781101968351026, 'businessWDL': 1.0570848609478811, 'businessDCN': 1.0804563918023302, 'businessNFM': 1.293179055678124, 'businessDeepFM': 1.0519467034898193, 'businessAFM': 1.083461280218211, 'businessxDeepFM': 1.1471864908

## Read Data

In [12]:
def Data2List(path):
    data = {}
    os.chdir(path)
    files = glob.glob('*.dat')
    file_name = [x[:-4] for x in files]
    for i in range(len(file_name)):
        data[file_name[i]] = pd.read_csv(files[i], sep = '\t', header = None)
        # print(file_name[i])
        # print(data[file_name[i]].head(2))
    return data

# Data Filtering
# delete users who's interaction less than 3
def del_less_than_3(data, target = 'user') :
    temp = data[target].value_counts()
    temp = temp.index[temp.values > 3]
    data = data[data[target].isin(temp)]
    
    return data

DoubanBook = Data2List('/home/rita/111/111-2MLG/HW3/data/DoubanBook')
Movielens = Data2List('/home/rita/111/111-2MLG/HW3/data/Movielens')
Yelp = Data2List('/home/rita/111/111-2MLG/HW3/data/Yelp')

DoubanBook['user_book'].columns = ['user', 'book', 'rating']
Movielens['user_movie'].columns = ["user","movie","rating","time"]
Yelp['user_business'].columns = ['user', 'business', 'rating']

DoubanBook['user_book'] = del_less_than_3(DoubanBook['user_book'])
Movielens['user_movie'] = del_less_than_3(Movielens['user_movie'])
Yelp['user_business'] = del_less_than_3(Yelp['user_business'])

# Merge user's feature
# DoubanBook['user'] = pd.merge(left = DoubanBook['user_location'], right = DoubanBook['user_group'], on = 0, how = 'outer')
# DoubanBook['user'].columns = ['user', 'location', 'group']
# DoubanBook['user'] = pd.merge(left = DoubanBook['user_book'], right = DoubanBook['user'], on = 'user', how = 'left')
DoubanBook['user'] = pd.merge(left = DoubanBook['user_book'], right = DoubanBook['user_location'], left_on = 'user', right_on = 0, how = 'left')
DoubanBook['user'] = DoubanBook['user'].drop([0], axis = 1)
DoubanBook['user'].columns = ['user', 'book', 'rating', 'location']

Movielens['user'] = pd.merge(left = Movielens['user_age'], right = Movielens['user_occupation'], on = 0, how = 'outer')
Movielens['user'].columns = ['user', 'age', 'occupation']
Movielens['user'] = pd.merge(left = Movielens['user_movie'], right = Movielens['user'], on = 'user', how = 'left')

Yelp['user'] = Yelp['user_business']   

print(DoubanBook['user'].shape)
print(DoubanBook['user'].head(2))
print(Movielens['user'].shape)
print(Movielens['user'].head(2))
print(Yelp['user'].shape)
print(Yelp['user'].head(2))
path = '/home/rita/111/111-2MLG/HW3'
os.chdir(path)

(788898, 4)
    user  book  rating  location
0  10855   938       4      33.0
1  10027     3       3     394.0
(100000, 6)
   user  movie  rating       time  age  occupation
0   196    242       3  881250949    5           3
1   186    302       3  891717742    4           4
(184835, 3)
   user  business  rating
2     2       186       5
3     2       205       5


In [13]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=3)
kf.get_n_splits(X)
print(kf)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


KFold(n_splits=3, random_state=None, shuffle=False)
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1 3] TEST: [2]
TRAIN: [0 1 2] TEST: [3]


## 10 Typical RecSys Methods
**UCF-s UCF-p ICF-s ICF-p** **MF** **FM** BPR-MF BPR-FM GBDT+LR XGB+LR 

In [100]:
# CF = Collaborative Filtering
# UCF-s / UCF-p / ICF-s / ICF-p
# https://elevenzou.github.io/2019/02/16/Surprise%E5%BA%93%E5%9F%BA%E7%A1%80%E7%94%A8%E6%B3%95/
# 4 mins
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.accuracy import rmse
def Based_CF(data, feats, based = 'U', sim = 's') :
    print('Start Training {}CF-{} model'.format(based, sim))
    reader = Reader(rating_scale=(1,5))
    data = Dataset.load_from_df(data[feats], reader)
    based_dict = {'U' : True, 'I' : False}
    sim_dict = {'s' : "cosine", 'p' : 'pearson'}
    sim_options = {"name":sim_dict[sim],"user-based":based_dict[based]}
    algo = KNNBasic(sim_options=sim_options)
    a = cross_validate(algo, data, cv=5, verbose=True, n_jobs = 5)
    print()
    return a['test_rmse'].mean()

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)

data_ls = [DoubanBook['user_book'], Movielens['user_movie'], Yelp['user_business']]
feat_ls = [['user', 'book', 'rating'], ["user", "movie", "rating"], ['user', 'business', 'rating']]
based = ['U', 'I']
sim = ['s', 'p']
for data, feats in zip(data_ls, feat_ls) :
    # print(feats)
    for b in based :
        for s in sim : 
            name = feats[1] + '{}CF-{}'.format(b, s)
            rmse_t = Based_CF(data, feats, b, s)
            rmse[name] = rmse_t

with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

Start Training UCF-s model
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7529  0.7576  0.7573  0.7601  0.7544  0.7565  0.0025  
MAE (testset)     0.6053  0.6089  0.6085  0.6105  0.6061  0.6079  0.0019  
Fit time          14.10   13.05   13.31   12.92   13.18   13.31   0.41    
Test time         19.75   19.49   19.81   19.65   19.75   19.69   0.11    

Start Training UCF-p model
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Co

In [16]:
# MF
# https://albertauyeung.github.io/2017/04/23/python-matrix-factorization.html/
# 9 mins
class Matrix_Factorization():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            # if (i+1) % 10 == 0:
            if (i+1) % 1 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

def MF(data, feats, target, K = 2, alpha = 0.01, beta = 0.01) :
    print(feats)
    train, test = train_test_split(data, test_size = config['test_size'])
    a = data[feats[0]].max()
    b = data[feats[1]].max()
    R = np.zeros((a, b), dtype = np.float128)
    train_x = np.array(train[feats])
    train_y = np.array(train[target])
    test_x = np.array(test[feats])
    test_y = np.array(test[target])
    for i in range(train_x.shape[0]):
        # R[train.iat[i, 0] - 1, train.iat[i, 1] - 1] = train.iat[i, 2]
        # R[train.at[i, feats[0]] - 1, train.at[i, feats[1]] - 1] = train.at[i, target]
        R[train_x[i, 0] - 1, train_x[i, 1] - 1] = train_y[i]
    

    # mf = Matrix_Factorization(R, K=K, alpha=alpha, beta=beta, iterations=config['n_epoch'])
    mf = Matrix_Factorization(R, K=K, alpha=alpha, beta=beta, iterations=2)
    training_process = mf.train()

    temp = mf.full_matrix()

    t = []
    for i in range(test.shape[0]) :
        t.append(temp[test_x[i, 0] - 1, test_x[i, 1] - 1])
    t = np.array(t)
    
    test_y = np.where(test_y > 3, 1, 0)
    t = np.where(t > 3, 1, 0)
    
    
    rmse = sqrt(mean_squared_error(test_y, t))
    ndcg = ndcg_score(test_y.reshape(1, -1), t.reshape(1, -1), k=10)
    recall = recall_score(test_y.reshape(1, -1), t.reshape(1, -1))
    
    print("test MSE : {:.4f}".format(rmse, 4), '\n')    
    
    return  rmse, ndcg, recall
    # print("Global bias:", mf.b, "User bias:", mf.b_u, "Item bias:", mf.b_i, sep = '\n')
    # x = [x for x, y in training_process]
    # y = [y for x, y in training_process]
    # plt.figure(figsize=((16,4)))
    # plt.plot(x, y)
    # plt.xticks(x, x)
    # plt.xlabel("Iterations")
    # plt.ylabel("Mean Square Error")
    # plt.grid(axis="y")
with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)
   
data_ls = [DoubanBook['user_book'], Movielens['user_movie'], Yelp['user_business']]
feats_ls = [['user', 'book', 'rating'], ["user", "movie", "rating"], ['user', 'business', 'rating']]
for data, feats in zip(data_ls, feats_ls):
    name = feats[1] + 'MF'
    rmse_t, ndcg_t, recall_t = MF(data, feats[:-1], feats[-1], K = 5, alpha = 0.1, beta = 0.15)
    # rmse[name] = rmse_t
    # ndcg_10[name] = ndcg_t
    # recall_10[name] = recall_t
    
with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)


['user', 'book']
Iteration: 1 ; error = 549.8098
Iteration: 2 ; error = 544.7074


ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

In [14]:
# FM
# https://github.com/coreylynch/pyFM
# 3 mins
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
def FM(data, feats, target) :
    print(feats)
    data = data.astype(str)
    train, test = train_test_split(data, test_size = config['test_size'])
    train_y = pd.Series(train[target].astype(float)).tolist()
    train_x = train[feats].to_dict('records')
    test_y = np.array(test[target].astype(float)) # .tolist()
    test_x = test[feats].to_dict('records')

    v = DictVectorizer()
    X = v.fit_transform(train_x).astype('double')
    fm = pylibfm.FM(num_factors=10, num_iter=config['n_epoch'], verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")
    fm.fit(X,train_y)
    temp = fm.predict(v.transform(test_x))

    rmse = sqrt(mean_squared_error(test_y, temp))
    test_y = np.where(test_y > 3, 1, 0)
    temp = np.where(temp > 3, 1, 0)
    ndcg = ndcg_score(test_y.reshape(1, -1), temp.reshape(1, -1), k=10)
    recall = recall_score(test_y, temp)
    print("test MSE : {:.4f}".format(rmse, 4), '\n')
    
    return rmse, ndcg, recall

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)
    
data_ls = [DoubanBook['user_book'], Movielens['user_movie'], Yelp['user_business']]
feats_ls = [['user', 'book', 'rating'], ["user", "movie", "rating"], ['user', 'business', 'rating']]
for data, feats in zip(data_ls, feats_ls):
    name = feats[1] + 'FM'
    rmse_t, ndcg_t, recall_t = FM(data, feats[:-1], feats[-1])
    rmse[name] = rmse_t
    ndcg_10[name] = ndcg_t
    recall_10[name] = recall_t
    
with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

['user', 'book']
Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.31979
-- Epoch 2
Training MSE: 0.29121
-- Epoch 3
Training MSE: 0.27841
-- Epoch 4
Training MSE: 0.27010
-- Epoch 5
Training MSE: 0.26407
-- Epoch 6
Training MSE: 0.25940
-- Epoch 7
Training MSE: 0.25563
-- Epoch 8
Training MSE: 0.25244
-- Epoch 9
Training MSE: 0.24976
-- Epoch 10
Training MSE: 0.24740
-- Epoch 11
Training MSE: 0.24533
-- Epoch 12
Training MSE: 0.24348
-- Epoch 13
Training MSE: 0.24178
-- Epoch 14
Training MSE: 0.24023
-- Epoch 15
Training MSE: 0.23884
-- Epoch 16
Training MSE: 0.23754
-- Epoch 17
Training MSE: 0.23632
-- Epoch 18
Training MSE: 0.23518
-- Epoch 19
Training MSE: 0.23410
-- Epoch 20
Training MSE: 0.23310
test MSE : 0.7036 

['user', 'movie']
Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.60366
-- Epoch 2
Training MSE: 0.52725
-- Epoch 3
Training MSE: 0.49865
-- Epoch 4
Training MSE

In [45]:
#not yet
# BPR-FM
# https://github.com/etlundquist/rankfm/blob/master/examples/movielens.ipynb
import os, sys, git
import numba as nb
import warnings

sns.set_style('whitegrid')
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
# plt.rcParams
# git_repo = git.Repo('.', search_parent_directories=True)
# git_root = git_repo.git.rev_parse('--show-toplevel')
# cython_path = os.path.join(git_root, 'rankfm')

# sys.path[0] = git_root
# sys.path[1] = cython_path
# sys.path[:2]
# !cd $git_root && python setup.py build_ext --inplace

# %load_ext autoreload
# %autoreload 2

import rankfm
from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity


# repo_root = "/Users/ericlundquist/Repos/rankfm"
# data_path = os.path.join(repo_root, "data/examples")
# print("\n".join([repo_root, data_path]))

interactions = Movielens['user_movie']
interactions_train, interactions_test = train_test_split(interactions, test_size = config['test_size'])

user_features = Movielens['user']
item_features = Movielens['movie_genre']

item_names = pd.read_csv(os.path.join(data_path, 'ML_1M_ITEM_NAMES.csv'))



unique_users = interactions.user_id.nunique()
unique_items = interactions.item_id.nunique()

print("interactions shape: {}".format(interactions.shape))
print("interactions unique users: {}".format(interactions.user_id.nunique()))
print("interactions unique items: {}".format(interactions.item_id.nunique()))

print("user features users:", interactions.user_id.nunique())
print("item features items:", interactions.item_id.nunique())



sparsity = 1 - (len(interactions) / (unique_users * unique_items))
print("interaction matrix sparsity: {}%".format(round(100 * sparsity, 1)))


np.random.seed(1492)
interactions['random'] = np.random.random(size=len(interactions))
test_pct = 0.25


train_mask = interactions['random'] <  (1 - test_pct)
valid_mask = interactions['random'] >= (1 - test_pct)

interactions_train = interactions[train_mask][['user_id', 'item_id']]
interactions_valid = interactions[valid_mask][['user_id', 'item_id']]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.item_id.unique())
valid_items = np.sort(interactions_valid.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

user_features = user_features[user_features.user_id.isin(train_users)]
item_features = item_features[item_features.item_id.isin(train_items)]
user_features.shape, item_features.shape

model = RankFM(factors=20, loss='warp', max_samples=20, alpha=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='invscaling')
model


%%time
model.fit(interactions_train, epochs=20, verbose=True)

valid_scores = model.predict(interactions_valid, cold_start='nan') 
print(valid_scores.shape)
pd.Series(valid_scores).describe()

valid_recommendations = model.recommend(valid_users, n_items=10, filter_previous=True, cold_start='nan')
valid_recommendations.head()

k = 10

most_popular = interactions_train.groupby('item_id')['user_id'].count().sort_values(ascending=False)[:k]
most_popular

test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_user_items = {key: val for key, val in test_user_items.items() if key in set(train_users)}

base_hrt = np.mean([int(len(set(most_popular.index) & set(val)) > 0)                       for key, val in test_user_items.items()])
base_pre = np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()])
base_rec = np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()])

print("number of test users: {}".format(len(test_user_items)))
print("baseline hit rate: {:.3f}".format(base_hrt))
print("baseline precision: {:.3f}".format(base_pre))
print("baseline recall: {:.3f}".format(base_rec))




%%time
model_hit_rate = hit_rate(model, interactions_valid, k=k)
model_reciprocal_rank = reciprocal_rank(model, interactions_valid, k=k)
model_dcg = discounted_cumulative_gain(model, interactions_valid, k=k)
model_precision = precision(model, interactions_valid, k=k)
model_recall = recall(model, interactions_valid, k=k)


print("hit_rate: {:.3f}".format(model_hit_rate))
print("reciprocal_rank: {:.3f}".format(model_reciprocal_rank))
print("dcg: {:.3f}".format(model_dcg, 3))
print("precision: {:.3f}".format(model_precision))
print("recall: {:.3f}".format(model_recall))

recommendation_diversity = diversity(model, interactions_valid, k=k)
recommendation_diversity.head(10)

top_items = pd.merge(item_names, recommendation_diversity, on='item_id', how='inner')
top_items = top_items.set_index('item_id').loc[recommendation_diversity.item_id].reset_index()
top_items = top_items[['item_id', 'cnt_users', 'pct_users', 'title', 'genres']]
top_items.head(10)

coverage = np.mean(recommendation_diversity['cnt_users'] > 0)
print("percentage of items recommended to at least one user: {:.3f}".format(coverage))

nonzero_users = recommendation_diversity[recommendation_diversity.cnt_users > 0]
entropy = -np.sum(nonzero_users['pct_users'] * np.log2(nonzero_users['pct_users']))
print("entropy value of recommended items: {:.3f}".format(entropy))

N = 50
fig, axes = plt.subplots(1, 1, figsize=[16, 4])

topN = recommendation_diversity.iloc[:N, :]
axes.bar(topN.index.values + 1, topN.pct_users, width=1, edgecolor='black', alpha=0.75)
axes.set(xlabel='Item Rank', ylabel='Percentage of Users', title='Percentage of Users Recommended by Item Rank')
plt.show()


random_user = np.random.choice(interactions_valid.user_id.unique())
print("random user: {}".format(random_user))

random_user_recs = valid_recommendations.loc[random_user]
random_user_recs = item_names[item_names.item_id.isin(random_user_recs)].set_index('item_id').loc[random_user_recs]
random_user_recs

most_similar_items = model.similar_items(919)
most_similar_items = item_names.set_index('item_id').loc[most_similar_items]
most_similar_items

most_similar_items = model.similar_items(2028)
most_similar_items = item_names.set_index('item_id').loc[most_similar_items]
most_similar_items



ImportError: cannot import name 'RankFM' from 'rankfm.rankfm' (/home/rita/111/111-2MLG/HW3/rankfm/rankfm/__init__.py)

## 10 NN-based RecSys Methods
FNN **IPNN** **OPNN** **PIN** **CCPM** NeuMF **WD** **DeepCross** **NFM** **DeepFM** 

In [8]:
#NN-based model
from deepctr_torch.models import PNN # PIN
from deepctr_torch.models import CCPM # CCPM

from deepctr_torch.models import WDL # WD
from deepctr_torch.models import DCN # DeepCross
from deepctr_torch.models import NFM # NFM
from deepctr_torch.models import DeepFM # DeepFM

#recent nn-based approach
from deepctr_torch.models import AFM # AFM
from deepctr_torch.models import xDeepFM # xDeePFM

# new



In [39]:

class nn_based :
    def __init__(self, data, features, target, device = 'cuda') : 
        self.features = features
        self.target = target
        for feat in self.features :
            enc = LabelEncoder()
            data[feat] = enc.fit_transform(data[feat])
        # SparseFeat(name, vocabulary_size, embedding_dim, use_hash, dtype, embedding_name, group_name)
        # Dense : Numeric 、 Sparse : Category 、 Sequence : Time Series       
        fixed = [SparseFeat(feat, data[feat].nunique()) for feat in self.features]
        linear_feats = fixed
        dnn_feats = fixed
        feature_names = get_feature_names(linear_feats + dnn_feats)

        # train test split
        self.train, self.test = train_test_split(data, test_size = 0.2)
        self.train_data = {name: self.train[name] for name in feature_names}
        self.test_data = {name: self.test[name] for name in feature_names}

        self.model_dict = {"IPNN":PNN(dnn_feats,use_inner=True,use_outter=False,task='regression',device=device),
              "OPNN":PNN(dnn_feats,use_inner=False,use_outter=True,task='regression',device=device),
              "PNN":PNN(dnn_feats,use_inner=True,use_outter=True,task='regression',device=device),
              "CCPM":CCPM(linear_feats, dnn_feats, task='regression',device=device),
             "WDL":WDL(linear_feats, dnn_feats, task='regression',device=device),
             "DCN":DCN(linear_feats, dnn_feats, task='regression',device=device),
             "NFM":NFM(linear_feats, dnn_feats, task='regression',device=device),
             "DeepFM":DeepFM(linear_feats, dnn_feats, task='regression',device=device),
             "AFM":AFM(linear_feats, dnn_feats, task='regression',device=device),
             "xDeepFM":xDeepFM(linear_feats, dnn_feats, task='regression',device=device), 
            #  ""
             }
             
    def train_model(self, model, n_epochs = 10):
        print(self.features, model, '==================', sep = '\n')
        model = self.model_dict[model]

        model.compile('adam', 'mse', metrics = ['mse'])
        history = model.fit(self.train_data,self.train[self.target].values,batch_size=256, epochs=n_epochs, verbose=5, validation_split=0.1)
        pred_ans = model.predict(self.test_data, batch_size=256)
        
        rmse = sqrt(mean_squared_error(self.test[self.target].values, pred_ans))
        
        test_y = np.where(self.test[self.target].values > 3, 1, 0)
        pred_ans = np.where(pred_ans > 3, 1, 0)
        
        ndcg = ndcg_score(test_y.reshape(1, -1), pred_ans.T, k=10)
        recall10 = recall_score(test_y.reshape(1, -1), pred_ans)
        
        print("test MSE : {:.4f}".format(rmse, 4))
        print("test ndcg : {:.4f}".format(ndcg, 4))
        # print("test recall10 : {:.4f}".format(recall10, 4))
        
        return rmse, ndcg, recall10


In [41]:
# 73 min
data_ls = [DoubanBook['user'], Movielens['user'], Yelp['user']]
feats_ls = [['user', 'book', 'location'], ['user', 'movie', 'time', 'age', 'occupation'], ['user', 'business']]
model = ['IPNN', 'OPNN', 'PNN', 'CCPM', 'WDL', 'DCN', 'NFM', 'DeepFM', 'AFM', 'xDeepFM']

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)    
    
for data, feats in zip(data_ls, feats_ls) :
    nn = nn_based(data, feats, 'rating')
    for m in model :
        rmse_t, ndcg_t, recall_t = nn.train_model(m, config['n_epoch'])
        name = feats[1] + m
        rmse[name] = rmse_t
        ndcg_10[name] = ndcg_t
        recall_10[name] = recall_t

with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

['user', 'book', 'location']
IPNN
cuda
Train on 568006 samples, validate on 63112 samples, 2219 steps per epoch


/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/deepctr_torch/layers/utils.py:61: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return [None if x is None else x[start:stop] for x in arrays]


Epoch 1/20
12s - loss:  0.7740 - mse:  0.7740 - val_mse:  0.4930
Epoch 2/20
12s - loss:  0.4764 - mse:  0.4764 - val_mse:  0.4803
Epoch 3/20
12s - loss:  0.4566 - mse:  0.4565 - val_mse:  0.4797
Epoch 4/20
12s - loss:  0.4419 - mse:  0.4418 - val_mse:  0.4849
Epoch 5/20
12s - loss:  0.4301 - mse:  0.4301 - val_mse:  0.4948
Epoch 6/20
13s - loss:  0.4189 - mse:  0.4189 - val_mse:  0.4957
Epoch 7/20
13s - loss:  0.4077 - mse:  0.4076 - val_mse:  0.5013
Epoch 8/20
14s - loss:  0.3988 - mse:  0.3987 - val_mse:  0.5130
Epoch 9/20
12s - loss:  0.3913 - mse:  0.3912 - val_mse:  0.5169
Epoch 10/20
13s - loss:  0.3847 - mse:  0.3846 - val_mse:  0.5196
Epoch 11/20
12s - loss:  0.3792 - mse:  0.3790 - val_mse:  0.5322
Epoch 12/20
13s - loss:  0.3746 - mse:  0.3745 - val_mse:  0.5349
Epoch 13/20
12s - loss:  0.3702 - mse:  0.3700 - val_mse:  0.5467
Epoch 14/20
12s - loss:  0.3657 - mse:  0.3655 - val_mse:  0.5508
Epoch 15/20
12s - loss:  0.3622 - mse:  0.3620 - val_mse:  0.5484
Epoch 16/20
13s - l

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/deepctr_torch/layers/utils.py:61: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return [None if x is None else x[start:stop] for x in arrays]


Epoch 1/20
13s - loss:  0.7865 - mse:  0.7865 - val_mse:  0.4905
Epoch 2/20
12s - loss:  0.4757 - mse:  0.4757 - val_mse:  0.4769
Epoch 3/20
13s - loss:  0.4550 - mse:  0.4550 - val_mse:  0.4772
Epoch 4/20
13s - loss:  0.4420 - mse:  0.4419 - val_mse:  0.4844
Epoch 5/20
13s - loss:  0.4304 - mse:  0.4304 - val_mse:  0.4818
Epoch 6/20
13s - loss:  0.4187 - mse:  0.4186 - val_mse:  0.4922
Epoch 7/20
14s - loss:  0.4076 - mse:  0.4075 - val_mse:  0.4939
Epoch 8/20
13s - loss:  0.3985 - mse:  0.3984 - val_mse:  0.4998
Epoch 9/20
13s - loss:  0.3917 - mse:  0.3916 - val_mse:  0.5058
Epoch 10/20
13s - loss:  0.3859 - mse:  0.3858 - val_mse:  0.5113
Epoch 11/20
13s - loss:  0.3800 - mse:  0.3799 - val_mse:  0.5224
Epoch 12/20
14s - loss:  0.3742 - mse:  0.3740 - val_mse:  0.5205
Epoch 13/20
13s - loss:  0.3692 - mse:  0.3690 - val_mse:  0.5283
Epoch 14/20
13s - loss:  0.3646 - mse:  0.3644 - val_mse:  0.5296
Epoch 15/20
14s - loss:  0.3604 - mse:  0.3602 - val_mse:  0.5386
Epoch 16/20
13s - l

...

Epoch 1/20
3s - loss:  1.6230 - mse:  1.6228 - val_mse:  1.0636
Epoch 2/20
3s - loss:  0.9607 - mse:  0.9607 - val_mse:  1.0540
Epoch 3/20
3s - loss:  0.9046 - mse:  0.9046 - val_mse:  1.0592
Epoch 4/20
3s - loss:  0.8799 - mse:  0.8799 - val_mse:  1.0649
Epoch 5/20
3s - loss:  0.8655 - mse:  0.8655 - val_mse:  1.0681
Epoch 6/20
3s - loss:  0.8560 - mse:  0.8559 - val_mse:  1.0829
Epoch 7/20
3s - loss:  0.8503 - mse:  0.8502 - val_mse:  1.0805
Epoch 8/20
3s - loss:  0.8455 - mse:  0.8455 - val_mse:  1.0910
Epoch 9/20
3s - loss:  0.8406 - mse:  0.8406 - val_mse:  1.0956
Epoch 10/20
3s - loss:  0.8368 - mse:  0.8368 - val_mse:  1.0911
Epoch 11/20
3s - loss:  0.8359 - mse:  0.8359 - val_mse:  1.0916
Epoch 12/20
3s - loss:  0.8320 - mse:  0.8320 - val_mse:  1.0978
Epoch 13/20
3s - loss:  0.8291 - mse:  0.8291 - val_mse:  1.0982
Epoch 14/20
3s - loss:  0.8276 - mse:  0.8276 - val_mse:  1.0955
Epoch 15/20
3s - loss:  0.8255 - mse:  0.8254 - val_mse:  1.0989
Epoch 16/20
3s - loss:  0.8240 - m

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/deepctr_torch/layers/utils.py:61: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return [None if x is None else x[start:stop] for x in arrays]


Epoch 1/20
3s - loss:  12.6600 - mse:  12.6591 - val_mse:  9.6370
Epoch 2/20
2s - loss:  6.5687 - mse:  6.5680 - val_mse:  4.2555
Epoch 3/20
2s - loss:  3.0539 - mse:  3.0536 - val_mse:  2.4993
Epoch 4/20
3s - loss:  1.9290 - mse:  1.9286 - val_mse:  1.8597
Epoch 5/20
3s - loss:  1.4631 - mse:  1.4628 - val_mse:  1.5635
Epoch 6/20
3s - loss:  1.2286 - mse:  1.2283 - val_mse:  1.4080
Epoch 7/20
3s - loss:  1.0955 - mse:  1.0953 - val_mse:  1.3180
Epoch 8/20
3s - loss:  1.0132 - mse:  1.0129 - val_mse:  1.2646
Epoch 9/20
3s - loss:  0.9590 - mse:  0.9587 - val_mse:  1.2311
Epoch 10/20
3s - loss:  0.9217 - mse:  0.9214 - val_mse:  1.2097
Epoch 11/20
3s - loss:  0.8951 - mse:  0.8948 - val_mse:  1.1957
Epoch 12/20
3s - loss:  0.8753 - mse:  0.8750 - val_mse:  1.1885
Epoch 13/20
3s - loss:  0.8603 - mse:  0.8599 - val_mse:  1.1842
Epoch 14/20
3s - loss:  0.8484 - mse:  0.8481 - val_mse:  1.1810
Epoch 15/20
3s - loss:  0.8388 - mse:  0.8384 - val_mse:  1.1807
Epoch 16/20
3s - loss:  0.8308 -

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/deepctr_torch/layers/utils.py:61: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return [None if x is None else x[start:stop] for x in arrays]


Epoch 1/20
4s - loss:  1.4309 - mse:  1.4308 - val_mse:  1.0604
Epoch 2/20
4s - loss:  0.9577 - mse:  0.9577 - val_mse:  1.0626
Epoch 3/20
4s - loss:  0.9020 - mse:  0.9020 - val_mse:  1.0642
Epoch 4/20
4s - loss:  0.8775 - mse:  0.8775 - val_mse:  1.0625
Epoch 5/20
4s - loss:  0.8608 - mse:  0.8608 - val_mse:  1.0768
Epoch 6/20
4s - loss:  0.8477 - mse:  0.8477 - val_mse:  1.0932
Epoch 7/20
4s - loss:  0.8374 - mse:  0.8374 - val_mse:  1.0996
Epoch 8/20
4s - loss:  0.8279 - mse:  0.8280 - val_mse:  1.0878
Epoch 9/20
4s - loss:  0.8182 - mse:  0.8182 - val_mse:  1.1025
Epoch 10/20
4s - loss:  0.8093 - mse:  0.8093 - val_mse:  1.1122
Epoch 11/20
4s - loss:  0.8009 - mse:  0.8009 - val_mse:  1.1117
Epoch 12/20
4s - loss:  0.7888 - mse:  0.7888 - val_mse:  1.1254
Epoch 13/20
4s - loss:  0.7757 - mse:  0.7757 - val_mse:  1.1460
Epoch 14/20
4s - loss:  0.7555 - mse:  0.7555 - val_mse:  1.1619
Epoch 15/20
4s - loss:  0.7290 - mse:  0.7290 - val_mse:  1.1926
Epoch 16/20
4s - loss:  0.6971 - m

##  3 Recent NN-based Methods
* **Attention Factorization Machines(AFM)**
* Collaborative Memory Networks (CMN)
* **xDeepFM**
* **Deep Interest Network (DIN)** *done for movielens*
* DeepGBM

In [167]:
#recent nn-based approach
from deepctr_torch.models import AFM
from deepctr_torch.models import xDeepFM
from deepctr_torch.models import DIN

# Din
import torch_rechub
from torch_rechub.utils.data import create_seq_features, df_to_dict, DataGenerator
from torch_rechub.basic.features import DenseFeature, SparseFeature, SequenceFeature
from torch_rechub.models.ranking import DIN
from torch_rechub.trainers import CTRTrainer


In [181]:

# Din
# need to have 'time' variable, so only movielens can be used
# https://www.cnblogs.com/junwei-kuang/p/DIN-DeepFM.html
# feature engeering
# 8 mins
# 7 epochs
data = Movielens['user_movie'].copy(deep = True)
data.columns = ['user_id', 'item_id', 'cate_id', 'time']


train, val, test = create_seq_features(data, seq_feature_col=['item_id', 'cate_id'], drop_short=3)
# get size of category data
n_users, n_items, n_cates = data["user_id"].max(), data["item_id"].max(), data["cate_id"].max()

features = [SparseFeature("target_item", vocab_size=n_items + 2, embed_dim=8), 
            SparseFeature("target_cate", vocab_size=n_cates + 2, embed_dim=8),
            SparseFeature("user_id", vocab_size=n_users + 2, embed_dim=8)]
target_features = features  
history_features = [
    SequenceFeature("history_item", vocab_size=n_items + 2, embed_dim=8, pooling="concat", shared_with="target_item"),
    SequenceFeature("history_cate", vocab_size=n_cates + 2, embed_dim=8, pooling="concat", shared_with="target_cate")]


train = df_to_dict(train)
val = df_to_dict(val)
test = df_to_dict(test)

train_y, val_y, test_y = train["label"], val["label"], test["label"]

# 删除 label 列
del train["label"]
del val["label"]
del test["label"]
train_x, val_x, test_x = train, val, test

# 构建dataloader，指定模型读取数据的方式，和区分验证集测试集、指定batch大小
dg = DataGenerator(train_x, train_y)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=val_x, y_val=val_y, x_test=test_x, y_test=test_y, batch_size=16)



# 定义模型，模型的参数需要我们之前的feature类，用于构建模型的输入层，mlp指定模型后续DNN的结构，attention_mlp指定attention层的结构
model = DIN(features=features, history_features=history_features, target_features=target_features, mlp_params={"dims": [256, 128]}, attention_mlp_params={"dims": [256, 128]})  # 这里的 features = target_features，理论上是不合理的，是为了特征太少来凑数？

# 模型训练，需要学习率、设备等一般的参数，此外我们还支持earlystoping策略，及时发现过拟合
ctr_trainer = CTRTrainer(model, optimizer_params={"lr": 1e-3, "weight_decay": 1e-3}, n_epoch=config['n_epoch'], earlystop_patience=4, device='cuda', model_path='./') 
ctr_trainer.fit(train_dataloader, val_dataloader)

# 查看在测试集上的性能
auc = ctr_trainer.evaluate(ctr_trainer.model, test_dataloader)
pred_y = np.array(ctr_trainer.predict(model, test_dataloader))
r = sqrt(mean_squared_error(test_y, pred_y))

test_y = np.where(test_y > 3, 1, 0)
pred_y = np.where(pred_y > 3, 1, 0)


n = ndcg_score(test_y.reshape(1, -1), pred_y.reshape(1, -1), k=10)
re = recall_score(test_y.reshape(1, -1), pred_y.reshape(1, -1))
name = 'movieDIN' 
print(f'test auc: {auc}')


    
with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)
    
rmse[name] = r
ndcg_10[name] = n
recall_10[name] = re

with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

epoch: 0


validation: 100%|██████████| 118/118 [00:01<00:00, 98.00it/s]


epoch: 0 validation: auc: 0.8392081408019576
epoch: 1


validation: 100%|██████████| 118/118 [00:01<00:00, 93.55it/s]


epoch: 1 validation: auc: 0.8839740050312116
epoch: 2


...

epoch: 18 validation: auc: 0.9676603516000581
epoch: 19


validation: 100%|██████████| 118/118 [00:01<00:00, 98.42it/s]


epoch: 19 validation: auc: 0.9652650719877109


predict: 100%|██████████| 118/118 [00:01<00:00, 96.21it/s]

test auc: 0.9763204681703326


## Results

In [ ]:
with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)


In [182]:
with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)


<class 'dict'>
49
{'bookIPNN': 0.7630446736162781, 'bookOPNN': 0.7596840523834419, 'bookPNN': 0.7518341524053948, 'bookCCPM': 0.7424491460611815, 'bookWDL': 0.7093089364684898, 'bookDCN': 0.7302810220618285, 'bookNFM': 0.7568314381827344, 'bookDeepFM': 0.7386060319249964, 'bookAFM': 0.7353113174115252, 'bookxDeepFM': 0.7524432051769194, 'movieIPNN': 1.1257715623148956, 'movieOPNN': 1.0723128454116133, 'moviePNN': 1.1457441626815474, 'movieCCPM': 1.053634406962656, 'movieWDL': 1.0492728323684728, 'movieDCN': 1.0530302980259754, 'movieNFM': 1.0942265873231904, 'movieDeepFM': 1.0515179004479969, 'movieAFM': 1.0710224032118236, 'moviexDeepFM': 1.0744275480292602, 'businessIPNN': 1.2039563606524462, 'businessOPNN': 1.1574377340755466, 'businessPNN': 1.1446729801067166, 'businessCCPM': 1.0781101968351026, 'businessWDL': 1.0570848609478811, 'businessDCN': 1.0804563918023302, 'businessNFM': 1.293179055678124, 'businessDeepFM': 1.0519467034898193, 'businessAFM': 1.083461280218211, 'businessxDee

In [4]:
iterables = [['book', 'movie', 'business'], ['rmse', 'recall', 'ndcg_10']]
# temp = pd.DataFrame(columns = ['book', 'movie', 'business'], index = model)
model = ['UCF-s', 'UCF-p', 'ICF-s', 'ICF-p', 'MF', 'FM', 'IPNN', 'OPNN', 'PNN', 'CCPM', 'WDL', 'DCN', 'NFM', 'DeepFM', 'AFM', 'DIN']
a = pd.MultiIndex.from_product(iterables, names = ['data', 'metrics'])
temp = pd.DataFrame(columns = a, index = model)
for i in temp.columns :
    for j in temp.index :
        for k in rmse.keys() :   
            if (i[0] in k) and (j in k) :
                t = sum([a.isupper() for a in k])
                if (t == 2) and (j == 'FM'): 
                    temp.at['FM', (i[0], 'rmse')] = round(rmse[k], 4)
                elif j != 'FM': 
                    # print(j)
                    temp.at[j, (i[0], 'rmse')] = round(rmse[k], 4)

for i in temp.columns :
    for j in temp.index :
        for k in ndcg_10.keys() :   
            if (i[0] in k) and (j in k) :
                t = sum([a.isupper() for a in k])
                if (t == 2) and (j == 'FM'): 
                    temp.at['FM', (i[0], 'ndcg_10')] = round(ndcg_10[k], 4)
                elif j != 'FM': 
                    # print(j)
                    temp.at[j, (i[0], 'ndcg_10')] = round(ndcg_10[k], 4)

print(temp)
# temp.to_csv('results.csv')
temp.to_csv('results_round4.csv', float_format='%.4f')

data       book                  movie                business               
metrics    rmse recall ndcg_10    rmse recall ndcg_10     rmse recall ndcg_10
UCF-s    0.7565    NaN     NaN  1.0173    NaN     NaN   1.1039    NaN     NaN
UCF-p    0.7739    NaN     NaN   1.012    NaN     NaN   1.1652    NaN     NaN
ICF-s    0.7563    NaN     NaN  1.0172    NaN     NaN   1.1049    NaN     NaN
ICF-p    0.7736    NaN     NaN   1.011    NaN     NaN    1.165    NaN     NaN
MF       0.7195    NaN   0.953  0.9534    NaN  0.8523   1.1082    NaN  0.8574
FM       0.7032    NaN  0.9786  0.9461    NaN  0.9739   1.0335    NaN  0.9584
IPNN      0.763    NaN   0.956  1.1258    NaN  0.8057    1.204    NaN    0.69
OPNN     0.7597    NaN  0.9086  1.0723    NaN   0.872   1.1574    NaN  0.6932
PNN      0.7518    NaN  0.8901  1.1457    NaN  0.8566   1.1447    NaN  0.8155
CCPM     0.7424    NaN  0.9575  1.0536    NaN  0.8787   1.0781    NaN  0.9271
WDL      0.7093    NaN  0.8775  1.0493    NaN  0.7686   1.0571  

In [5]:
temp = pd.read_csv('results_round4.csv', header=[0,1], index_col = [0])
print(temp)

data       book                  movie                business               
metrics    rmse recall ndcg_10    rmse recall ndcg_10     rmse recall ndcg_10
UCF-s    0.7565    NaN     NaN  1.0173    NaN     NaN   1.1039    NaN     NaN
UCF-p    0.7739    NaN     NaN  1.0120    NaN     NaN   1.1652    NaN     NaN
ICF-s    0.7563    NaN     NaN  1.0172    NaN     NaN   1.1049    NaN     NaN
ICF-p    0.7736    NaN     NaN  1.0110    NaN     NaN   1.1650    NaN     NaN
MF       0.7195    NaN  0.9530  0.9534    NaN  0.8523   1.1082    NaN  0.8574
FM       0.7032    NaN  0.9786  0.9461    NaN  0.9739   1.0335    NaN  0.9584
IPNN     0.7630    NaN  0.9560  1.1258    NaN  0.8057   1.2040    NaN  0.6900
OPNN     0.7597    NaN  0.9086  1.0723    NaN  0.8720   1.1574    NaN  0.6932
PNN      0.7518    NaN  0.8901  1.1457    NaN  0.8566   1.1447    NaN  0.8155
CCPM     0.7424    NaN  0.9575  1.0536    NaN  0.8787   1.0781    NaN  0.9271
WDL      0.7093    NaN  0.8775  1.0493    NaN  0.7686   1.0571  